In [ ]:
print("Running")

In [ ]:
import gym
import glob
import torch
import matplotlib.pyplot as plt

In [ ]:
deg2rad = lambda x: x * 3.1415926536 / 180.
rad2deg = lambda x: x * 180. / 3.1415926536

In [ ]:
env = gym.make('gym_swirl:swirl-v1')
env.seed(42)

In [ ]:
set([p.split("mfmaac").pop().split("_train")[0] for p in glob.glob("runs/mfmaac*")])

In [ ]:
paths = sorted(glob.glob("runs/mfmaac068_*"), key=lambda x: int(x.split("epoch").pop().split("_2021")[0]))
#path = paths[-1]
len(paths)

In [ ]:
hist = torch.zeros(180)
hist_amount = torch.zeros(180)
for path in paths[-10::]:
    env.load(path)
    for state in env.states[1:]:
        indexes = rad2deg(state.Deltas).type(torch.int64)
        #print("orig size",indexes.size())
        #print("indexes", indexes.min(), indexes.max())
        valid = (indexes >= 0) & (indexes < 180) 
        #print("valid sum", valid.sum())
        indexes = indexes[valid]
        #print("new index", indexes.size())
        ors = state.O_R[valid]#.abs()
        #print("ors", ors.size())
        #print("hist", hist.size())
        hist.index_add_(0, indexes, ors)
        #print("hist_amount", hist_amount.size())
        #print("torch.ones_like(indexes)", torch.ones_like(indexes).size())
        hist_amount.index_add_(0, indexes, torch.ones_like(indexes).type(torch.float32))

plt.figure(figsize=(15, 10))
plt.plot((hist/torch.maximum(hist_amount, torch.tensor([1.]))).abs())

In [ ]:
(hist/torch.maximum(hist_amount, torch.tensor([1.]))).abs().argmax()